In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Sergey Grishaev Link Prediction app").getOrCreate()

In [3]:
spark

In [4]:
graphPath = "/lectures/lecture04/trainGraph/"
usersToPredictPath = "/lectures/lecture04/prediction.csv"

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType(fields=[
    StructField("user", IntegerType()),
    StructField("friendsString", StringType())
])

data = spark.read.format("csv") \
        .schema(schema) \
        .option("delimiter", "\t") \
        .load(graphPath).repartition(15)

In [6]:
data.show(5)

+--------+--------------------+
|    user|       friendsString|
+--------+--------------------+
| 9305887|{(1528306,64),(17...|
|11564353|{(494256,0),(6465...|
| 1155617|{(32646,0),(59257...|
|  160974|{(16743,0),(20215...|
| 7195016|{(1286906,0),(177...|
+--------+--------------------+
only showing top 5 rows



In [7]:
data.rdd.getNumPartitions()

15

In [8]:
from pyspark.sql.functions import col, explode, collect_list, sort_array, size, split, lit


In [9]:
from pyspark.sql.functions import pandas_udf

def cutStartEndBrackets(series):
    return series.str[2:-2]

cutStartEndBracketsUDF = pandas_udf(cutStartEndBrackets, StringType())

userFriend = \
    data.select(col("user"), split(cutStartEndBracketsUDF(col("friendsString")), "\),\(").alias("friendsMasks"))\
    .withColumn("friendMask", explode('friendsMasks'))\
    .withColumn("friend", split(col("friendMask"), ",")[0])\
    .select(col("user"), col("friend").cast("integer"))

<img src="pics/step1.jpg" width=700/>

In [10]:
userFriend.show()

+------+------+
|  user|friend|
+------+------+
|673670|  7404|
|673670| 16264|
|673670| 34059|
|673670| 38798|
|673670| 44596|
|673670| 64153|
|673670| 75927|
|673670|102190|
|673670|133880|
|673670|146923|
|673670|151239|
|673670|174052|
|673670|182919|
|673670|202396|
|673670|223253|
|673670|227570|
|673670|236396|
|673670|253426|
|673670|284262|
|673670|327163|
+------+------+
only showing top 20 rows



In [11]:
usersWithCommonFriend = userFriend\
    .groupBy("friend")\
    .agg(collect_list("user").alias("usersWithCommonFriend")) \
    .select("usersWithCommonFriend")\
    .where(size(col("usersWithCommonFriend")) >= 2)\
    .select(sort_array("usersWithCommonFriend").alias("sortedUsersWithCommonFriend"))\
    .drop("usersWithCommonFriend")

<img src="pics/step2.jpg" width=700/>


In [12]:
usersWithCommonFriend.show()

+---------------------------+
|sortedUsersWithCommonFriend|
+---------------------------+
|          [131318, 2038934]|
|       [11979968, 11979968]|
|       [4471643, 4864911...|
|           [39812, 3996243]|
|       [34660, 960407, 1...|
|         [215426, 10989590]|
|       [294833, 3541819,...|
|           [585940, 595464]|
|         [7941296, 7941296]|
|       [213875, 3310959,...|
|       [1406846, 2044978...|
|       [1208384, 1208384...|
|        [2625623, 16545603]|
|        [9787095, 11510201]|
|       [1093525, 1543499...|
|       [10599936, 10599936]|
|       [5021715, 5262480...|
|        [1651903, 10655397]|
|       [1919, 582013, 35...|
|         [3243642, 5845843]|
+---------------------------+
only showing top 20 rows



In [13]:
schema_users_to_pred = StructType(fields=[
    StructField("user", IntegerType()),
])

usersToPredict = spark.read.format("csv") \
    .schema(schema_users_to_pred) \
    .load(usersToPredictPath) \
    .select(col("user").cast("integer")) \
    .rdd.map(lambda t : t.user).collect()

usersToPredictBC = spark.sparkContext.broadcast(set(usersToPredict))

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType

def pairsWithCommonFriend(usersWithCommonFriend):
    pairs = []
    for user1Index in range(0, len(usersWithCommonFriend)):
        for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
            if user1Index != user2Index:
                if (usersWithCommonFriend[user1Index] in usersToPredictBC.value or \
                usersWithCommonFriend[user2Index] in usersToPredictBC.value):
                    pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
    return pairs

schema = ArrayType(ArrayType(IntegerType()))

pairsWithCommonFriendUdf = udf(pairsWithCommonFriend, schema)

<img src="pics/step4_2.jpg" width=700/>

In [15]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

def pairsWithCommonFriend(series):
    pairs_lists = []
    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
            for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                if user1Index != user2Index:
                    if usersWithCommonFriend[user1Index] in usersToPredictBC.value or \
                    usersWithCommonFriend[user2Index] in usersToPredictBC.value:
                        pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)
        
pairsWithCommonFriendUdf = pandas_udf(pairsWithCommonFriend, schema)

commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdf("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0).repartition(20)    

commonFriendsCounts\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(20)

+--------------------+-----+
|pairWithCommonFriend|count|
+--------------------+-----+
|     [179, 10958460]|   23|
|      [369, 8230185]|    5|
|      [1563, 737128]|    1|
|    [1583, 15110410]|   22|
|     [2405, 1029073]|    1|
|     [2405, 3414844]|    2|
|     [2405, 3520659]|    1|
|     [2405, 3739978]|    1|
|     [2405, 3937568]|    4|
|    [2405, 12274110]|    9|
|    [2405, 12889361]|    7|
|    [2405, 14371435]|    5|
|    [2405, 15124711]|    8|
|    [2405, 16425948]|    2|
|     [4151, 9576530]|    4|
|    [4151, 16519433]|    2|
|     [5023, 2038594]|   15|
|     [5023, 2474666]|   10|
|     [5023, 5029047]|   21|
|    [5023, 16425380]|    4|
+--------------------+-----+
only showing top 20 rows



In [ ]:
def pairsWithCommonFriend(series):
    pairs_lists = []
    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
            for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                if user1Index != user2Index:
                    pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)
         
pairsWithCommonFriendUdf = pandas_udf(pairsWithCommonFriend, schema_pandas)

commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdf("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0)    

commonFriendsCounts\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(20)

In [ ]:
from functools import partial

def pairsWithCommonFriendUpgraded(series, modulo):
    pairs_lists = []

    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
             for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                    if user1Index != user2Index and user1Index % 13 == modulo:
                        pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)


for i in range(13):
    pairsWithCommonFriendUdfUpgraded = pandas_udf(partial(pairsWithCommonFriendUpgraded, modulo=i), schema)

    commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdfUpgraded("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0)\
            .write.parquet("pairs/" + str(i), mode = "overwrite")

In [ ]:
spark.read.parquet("pairs/*").count()

In [ ]:
spark.read.parquet("pairs/0").count()

In [ ]:
spark.read.parquet("pairs/*")\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(5)

In [ ]:
spark.stop()